In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

In [2]:
bs = 8
torch.cuda.set_device(0)

In [3]:
data_path = Config.data_path()
data_path

WindowsPath('C:/Users/NANI.HOMIES/.fastai/data')

In [4]:
lang = 'mr'

In [5]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

In [6]:
path

WindowsPath('C:/Users/NANI.HOMIES/.fastai/data/mrwiki')

In [7]:
from nlpp import get_wiki

In [8]:
get_wiki(path,lang)

C:\Users\NANI.HOMIES\.fastai\data\mrwiki\mrwiki already exists; not downloading


In [9]:
# shutil.move(str(path/'text/AA/wiki_00'), str(path/name))
# shutil.rmtree(path/'text')

In [10]:
from nlpu import split_wiki

In [11]:
dest = split_wiki(path,lang)

C:\Users\NANI.HOMIES\.fastai\data\mrwiki\docs already exists; not splitting


In [12]:

dest.ls()[:5]

[WindowsPath('C:/Users/NANI.HOMIES/.fastai/data/mrwiki/docs/mr_databunch'),
 WindowsPath('C:/Users/NANI.HOMIES/.fastai/data/mrwiki/docs/अंकगणित.txt'),
 WindowsPath('C:/Users/NANI.HOMIES/.fastai/data/mrwiki/docs/अथर्ववेद.txt'),
 WindowsPath('C:/Users/NANI.HOMIES/.fastai/data/mrwiki/docs/अभियांत्रिकी.txt'),
 WindowsPath('C:/Users/NANI.HOMIES/.fastai/data/mrwiki/docs/अमोल पालेकर.txt')]

In [13]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

(3128, 87)

In [14]:
data = load_data(path, f'{lang}_databunch', bs=bs)

In [15]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [16]:

lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

In [ ]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

In [ ]:
#Sentiment Analysis

In [ ]:
train_df = pd.read_csv(path/'train.csv')
train_df.loc[pd.isna(train_df.comment),'comment']='NA'
train_df.head()

In [ ]:
test_df = pd.read_csv(path/'test.csv')
test_df.loc[pd.isna(test_df.comment),'comment']='NA'
test_df.head()

In [ ]:
df = pd.concat([train_df,test_df], sort=False)

In [ ]:
data_lm = (TextList.from_df(df, path, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [ ]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [ ]:
lr = 1e-3
lr *= bs/48

In [ ]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

In [ ]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

In [ ]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='label')
    .databunch(bs=bs, num_workers=1))

data_clas.save(f'{lang}_textlist_class')

In [ ]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)

In [ ]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr=2e-2
lr *= bs/92

In [ ]:

learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:

# learn_c.freeze_to(-2)
# learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr/2), moms=(0.8,0.7))

In [ ]:
import torch
t_cpu = torch.rand(500,500,500)
%timeit t_cpu @ t_cpu